In [3]:
import cv2
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Load MoveNet model
movenet = hub.load('https://tfhub.dev/google/movenet/singlepose/thunder/4')

# Function to calculate the angle between two points
def calculate_angle(point1, point2, point3):
    a = np.array(point1)  # First point (shoulder)
    b = np.array(point2)  # Middle point (hip)
    c = np.array(point3)  # Last point (knee)
    
    # Calculate the angle
    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians * 180.0 / np.pi)
    
    if angle > 180.0:
        angle = 360 - angle
        
    return angle

# Update this to your correct input video path
video_path = r"G:\Pose AI\Input Video\WhatsApp Video 2024-09-18 at 10.22.46_d6797a9c.mp4"

# Process the video
cap = cv2.VideoCapture(video_path)

# Check if video opened successfully
if not cap.isOpened():
    print("Error: Could not open video. Please check the video path.")
    exit()

# Initialize variables
frame_num = 0
angle_data = []

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("End of video or error reading frame.")
        break
    
    # Resize frame to 256x256 pixels (required input size for MoveNet)
    img = cv2.resize(frame, (256, 256))
    img = img.astype(np.float32)
    img = np.expand_dims(img, axis=0)
    
    # Normalize the image
    img = (img / 255.0).astype(np.float32)

    # Run the MoveNet model
    outputs = movenet(tf.constant(img))

    # Extract keypoints (landmarks)
    keypoints = outputs['output_0'].numpy()[0, 0, :, :]

    # Check if keypoints were extracted correctly
    if keypoints is None or keypoints.size == 0:
        print(f"Frame {frame_num}: No keypoints found.")
        continue

    # Get landmarks for upper and lower body
    try:
        left_shoulder = keypoints[5][:2]  # Left shoulder
        left_hip = keypoints[11][:2]      # Left hip
        left_knee = keypoints[13][:2]     # Left knee
    except IndexError as e:
        print(f"Error extracting keypoints for frame {frame_num}: {e}")
        continue

    # Calculate the angle between upper and lower body (e.g., shoulder-hip-knee)
    angle = calculate_angle(left_shoulder, left_hip, left_knee)

    # Append the angle to the list with the current frame number
    angle_data.append({'frame': frame_num, 'angle': angle})
    
    # Print debug information for verification
    print(f"Frame {frame_num}: Angle {angle}")
    
    frame_num += 1

cap.release()

# Check if angle_data has been populated correctly
if not angle_data:
    print("No data collected. Please check the video or the keypoint extraction process.")
else:
    print("Data collected successfully.")

# Only create DataFrame and plot if data exists
if angle_data:
    # Convert the angle data to a DataFrame
    angles_df = pd.DataFrame(angle_data)

    # Convert 'frame' and 'angle' columns to numeric types (if necessary)
    angles_df['frame'] = pd.to_numeric(angles_df['frame'], errors='coerce')
    angles_df['angle'] = pd.to_numeric(angles_df['angle'], errors='coerce')

    # Save the data to a CSV file
    angles_df.to_csv('angle_data_output.csv', index=False)

    # Plot the angle vs. frame number
    plt.plot(angles_df['frame'], angles_df['angle'])
    plt.xlabel('Frame')
    plt.ylabel('Angle (degrees)')
    plt.title('Angle Over Time')
    plt.show()


TypeError: '_UserObject' object is not callable